## Setup

In [1]:
from google.colab import files
import json
# files.upload()

In [2]:
!rm -r ~/.kaggle
!mkdir ~/.kaggle
%cd ~/.kaggle/
!touch kaggle.json
api_token = {"username":"avibewtra","key":"57c0b1a4dd936e5f3e20c684b854e472"}
with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)
!chmod 600 ~/.kaggle/kaggle.json
%cd


rm: cannot remove '/root/.kaggle': No such file or directory
/root/.kaggle
/root


In [3]:
!cat ~/.kaggle/kaggle.json

{"username": "avibewtra", "key": "57c0b1a4dd936e5f3e20c684b854e472"}

In [4]:
# https://www.kaggle.com/datasets/erichqiu/nba-odds-and-scores
!mkdir nba-data
%cd nba-data
!kaggle datasets download -d erichqiu/nba-odds-and-scores

/root/nba-data
  0% 0.00/1.87M [00:00<?, ?B/s]
100% 1.87M/1.87M [00:00<00:00, 178MB/s]


In [5]:
!unzip -q nba-odds-and-scores.zip

## Pandas

In [9]:
import pandas as pd
import numpy as np

In [10]:
%cd 2012-13
!ls

[Errno 2] No such file or directory: '2012-13'
/root/nba-data/2012-13
raw_scores.txt	vegas_playoff.txt  vegas.txt


In [11]:
raw_scores = pd.read_csv("raw_scores.txt")
vegas = pd.read_csv("vegas.txt")
vegas_p = pd.read_csv("vegas_playoff.txt")

In [12]:
teams_odds = pd.merge(raw_scores, vegas, left_on=["GAME_ID", "TEAM_ID"], right_on=["GameId", "TeamId"])

In [13]:
teams_odds_moneyline = teams_odds[['TeamId', 'GameId', 'PTS_QTR1', 'PTS_QTR2', 'PTS_QTR3', 'PTS_QTR4','PTS', 'FG_PCT', 'FT_PCT', 'FG3_PCT', 'AST', 'REB', 'TOV', 'PercentBet_ML', 'Open_Line_ML', 'Pinnacle_ML', '5dimes_ML', 'Heritage_ML', 'Bovada_ML', 'Betonline_ML', 'Average_Line_ML', 'Best_Line_ML', 'Worst_Line_ML']]

In [ ]:
columns

## Data Perturbations

### Row Uniformity
Take out a percentage of the data and replace the removed rows with random duplicates

In [ ]:
def add_duplicate_rows(df, p=.25, reset_index=False):
  df2 = pd.concat([df, df.sample(n=int(len(df)*p))])

  if reset_index: 
    return df2.reset_index(drop=True)
  else: 
    return df2
    
add_duplicate_rows(teams_odds_moneyline, p=.25)

,TeamId,GameId,PTS_QTR1,PTS_QTR2,PTS_QTR3,PTS_QTR4,PTS,FG_PCT,FT_PCT,FG3_PCT,...,PercentBet_ML,Open_Line_ML,Pinnacle_ML,5dimes_ML,Heritage_ML,Bovada_ML,Betonline_ML,Average_Line_ML,Best_Line_ML,Worst_Line_ML
0,1610612764,21200001,24,15,23,22,84,0.356,0.600,0.250,...,44.6,220.0,210.0,200,210,215.0,210.0,209.00,215.0,200.0
1,1610612739,21200001,31,19,24,20,94,0.456,0.682,0.350,...,55.4,-260.0,-235.0,-240,-250,-255.0,-250.0,-246.00,-235.0,-255.0
2,1610612738,21200002,25,29,22,31,107,0.520,0.821,0.462,...,44.8,220.0,244.0,240,230,225.0,235.0,234.80,244.0,225.0
3,1610612748,21200002,31,31,31,27,120,0.544,0.813,0.500,...,55.2,-260.0,-275.0,-260,-270,-265.0,-280.0,-270.00,-260.0,-280.0
4,1610612742,21200003,25,23,26,25,99,0.471,0.778,0.333,...,40.8,350.0,393.0,420,400,375.0,385.0,394.60,420.0,375.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1791,1610612755,21200896,23,23,28,27,101,0.410,0.778,0.238,...,48.7,115.0,121.0,125,125,130.0,130.0,126.20,130.0,121.0
2185,1610612758,21201093,37,20,18,23,98,0.453,0.813,0.292,...,51.0,100.0,118.0,125,120,125.0,120.0,121.60,125.0,118.0
339,1610612766,21200171,23,28,23,17,91,0.373,0.808,0.364,...,50.0,140.0,142.0,145,140,140.0,140.0,141.40,145.0,140.0
1893,1610612744,21200947,26,24,25,17,92,0.410,0.708,0.550,...,61.4,-165.0,-132.0,-130,-140,NaN,-135.0,-134.25,-130.0,-140.0


### Timeliness
Predict a years/games odds based on data from two years ago, not the year prior—or something to this effect.




In [ ]:
# Use teams_odds_moneyline to predict 2013-14 odds

### Change the data
Add noise


In [ ]:
def add_col_noise(df, ignore_cols=['TeamId', 'GameId']):
  for col in df.columns.to_list():
    if col in ignore_cols:
      continue
    else:
      std = df[col].std()
      noise = np.random.normal(0, std, len(df[col]))
      df[col] += noise
  return df

add_col_noise(teams_odds_moneyline)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,TeamId,GameId,PTS_QTR1,PTS_QTR2,PTS_QTR3,PTS_QTR4,PTS,FG_PCT,FT_PCT,FG3_PCT,...,PercentBet_ML,Open_Line_ML,Pinnacle_ML,5dimes_ML,Heritage_ML,Bovada_ML,Betonline_ML,Average_Line_ML,Best_Line_ML,Worst_Line_ML
0,1610612764,21200001,10.293375,22.537824,-1.364863,11.406169,174.003501,0.530536,0.648656,0.332724,...,45.611052,3514.371864,1699.074305,-210.112929,1458.228510,258.655780,3471.734312,813.082940,776.776290,3583.730336
1,1610612739,21200001,37.021693,15.981810,35.944111,5.745446,160.794322,-0.005324,0.886194,0.641551,...,393.530896,-2338.309066,-787.716744,-2946.343777,2036.393760,1231.397356,1496.526726,2306.983080,-192.684590,3570.247145
2,1610612764,21200001,28.015540,30.814601,57.088938,-2.687533,144.803721,0.212536,1.413611,0.596647,...,83.057734,1895.806529,-1041.326969,1124.146463,1501.151577,2978.842686,3028.913705,3280.217602,2016.120171,27.462155
3,1610612739,21200001,42.159204,33.463504,27.333710,10.740360,50.341990,0.825939,0.120148,0.410284,...,29.680449,-521.014686,-2452.443686,-428.254613,-222.124898,-488.608171,149.310087,2555.141094,819.645586,-1497.105481
4,1610612738,21200002,62.318602,59.894988,68.017084,-12.554407,113.317034,0.207248,0.840767,0.296346,...,107.485878,1457.358992,723.745102,-186.540360,3949.058705,904.700288,2011.637789,1647.065923,854.360440,3396.623462
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4911,1610612758,21201229,48.698494,22.290303,52.527132,59.738946,101.220629,0.840607,0.876264,0.159166,...,115.069055,884.248209,-530.270404,-1027.617786,420.199499,911.112476,588.057556,2776.750797,2618.260930,-2346.017472
4912,1610612744,21201230,5.358498,33.391399,53.744451,0.415191,104.274150,0.028940,1.347471,0.653218,...,-36.927668,-1365.381212,866.166375,159.056561,2190.619005,641.435834,-2701.998961,-1496.659307,-3301.654393,-2910.654771
4913,1610612757,21201230,48.520786,18.658315,36.381507,57.191997,126.172728,0.379570,1.502680,-0.035232,...,72.976716,941.157567,2074.187342,1151.247883,-710.791493,571.940697,523.350905,-610.349813,945.773586,-63.383898
4914,1610612744,21201230,9.678046,4.404986,21.860782,-19.101627,45.467029,0.552892,0.498168,0.918755,...,219.439253,-641.152454,773.303151,-417.224507,990.195622,699.714084,150.916610,-1677.641396,-2417.270303,-3393.845387


### Feature Statistics: Create synthetic data
- Using the statistics of each feature: mean, variance, min, max, etc; or use another probabilistic model.
- Using changed statistics of each feature: mean, variance, min, max, etc; or use another probabilistic model.


In [ ]:
def bootstrapped_df(df, keep_cols=['TeamId', 'GameId']):
  df2 = pd.DataFrame()
  for col in df.columns.to_list():
    if col in keep_cols:
      df2[col] = df[col]
    else:
      mu = df[col].mean()
      std = df[col].std()
      noise = np.random.normal(mu, std**.5, len(df[col]))
      df2[col] = noise

  return df2

bootstrapped_df(teams_odds_moneyline)

### Drop features




In [ ]:
def drop_features(df, n_cols=2, keep_columns=['TeamId', 'GameId']):
  columns = df.drop(keep_columns, axis=1).columns.to_list()
  drop = np.random.choice(columns, size=n_cols)
  return df.drop(drop, axis=1)

drop_features(teams_odds_moneyline)

,TeamId,GameId,PTS_QTR1,PTS_QTR2,PTS_QTR3,PTS_QTR4,PTS,FG_PCT,FT_PCT,AST,...,TOV,PercentBet_ML,Open_Line_ML,Pinnacle_ML,5dimes_ML,Heritage_ML,Bovada_ML,Betonline_ML,Best_Line_ML,Worst_Line_ML
0,1610612764,21200001,10.293375,22.537824,-1.364863,11.406169,174.003501,0.530536,0.648656,49.926409,...,-2.596967,45.611052,3514.371864,1699.074305,-210.112929,1458.228510,258.655780,3471.734312,776.776290,3583.730336
1,1610612739,21200001,37.021693,15.981810,35.944111,5.745446,160.794322,-0.005324,0.886194,23.617827,...,8.094681,393.530896,-2338.309066,-787.716744,-2946.343777,2036.393760,1231.397356,1496.526726,-192.684590,3570.247145
2,1610612764,21200001,28.015540,30.814601,57.088938,-2.687533,144.803721,0.212536,1.413611,-4.868736,...,31.883002,83.057734,1895.806529,-1041.326969,1124.146463,1501.151577,2978.842686,3028.913705,2016.120171,27.462155
3,1610612739,21200001,42.159204,33.463504,27.333710,10.740360,50.341990,0.825939,0.120148,33.774146,...,28.926318,29.680449,-521.014686,-2452.443686,-428.254613,-222.124898,-488.608171,149.310087,819.645586,-1497.105481
4,1610612738,21200002,62.318602,59.894988,68.017084,-12.554407,113.317034,0.207248,0.840767,31.850634,...,5.201117,107.485878,1457.358992,723.745102,-186.540360,3949.058705,904.700288,2011.637789,854.360440,3396.623462
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4911,1610612758,21201229,48.698494,22.290303,52.527132,59.738946,101.220629,0.840607,0.876264,52.018686,...,3.131088,115.069055,884.248209,-530.270404,-1027.617786,420.199499,911.112476,588.057556,2618.260930,-2346.017472
4912,1610612744,21201230,5.358498,33.391399,53.744451,0.415191,104.274150,0.028940,1.347471,21.119084,...,-1.731481,-36.927668,-1365.381212,866.166375,159.056561,2190.619005,641.435834,-2701.998961,-3301.654393,-2910.654771
4913,1610612757,21201230,48.520786,18.658315,36.381507,57.191997,126.172728,0.379570,1.502680,32.548719,...,15.889344,72.976716,941.157567,2074.187342,1151.247883,-710.791493,571.940697,523.350905,945.773586,-63.383898
4914,1610612744,21201230,9.678046,4.404986,21.860782,-19.101627,45.467029,0.552892,0.498168,26.024716,...,-12.786885,219.439253,-641.152454,773.303151,-417.224507,990.195622,699.714084,150.916610,-2417.270303,-3393.845387


### Add dummy features

In [ ]:
def add_dummy_cols(df, n_cols=2):
  for i in range(n_cols):
    df[i + 10000] = np.random.normal(size=len(df))
  return df

add_dummy_cols(teams_odds_moneyline)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,TeamId,GameId,PTS_QTR1,PTS_QTR2,PTS_QTR3,PTS_QTR4,PTS,FG_PCT,FT_PCT,FG3_PCT,...,Pinnacle_ML,5dimes_ML,Heritage_ML,Bovada_ML,Betonline_ML,Average_Line_ML,Best_Line_ML,Worst_Line_ML,10000,10001
0,1610612764,21200001,10.293375,22.537824,-1.364863,11.406169,174.003501,0.530536,0.648656,0.332724,...,1699.074305,-210.112929,1458.228510,258.655780,3471.734312,813.082940,776.776290,3583.730336,1.156746,-2.079561
1,1610612739,21200001,37.021693,15.981810,35.944111,5.745446,160.794322,-0.005324,0.886194,0.641551,...,-787.716744,-2946.343777,2036.393760,1231.397356,1496.526726,2306.983080,-192.684590,3570.247145,0.407127,1.397979
2,1610612764,21200001,28.015540,30.814601,57.088938,-2.687533,144.803721,0.212536,1.413611,0.596647,...,-1041.326969,1124.146463,1501.151577,2978.842686,3028.913705,3280.217602,2016.120171,27.462155,2.442481,0.267886
3,1610612739,21200001,42.159204,33.463504,27.333710,10.740360,50.341990,0.825939,0.120148,0.410284,...,-2452.443686,-428.254613,-222.124898,-488.608171,149.310087,2555.141094,819.645586,-1497.105481,0.801579,0.200581
4,1610612738,21200002,62.318602,59.894988,68.017084,-12.554407,113.317034,0.207248,0.840767,0.296346,...,723.745102,-186.540360,3949.058705,904.700288,2011.637789,1647.065923,854.360440,3396.623462,0.025599,-1.121344
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4911,1610612758,21201229,48.698494,22.290303,52.527132,59.738946,101.220629,0.840607,0.876264,0.159166,...,-530.270404,-1027.617786,420.199499,911.112476,588.057556,2776.750797,2618.260930,-2346.017472,0.933341,-0.767366
4912,1610612744,21201230,5.358498,33.391399,53.744451,0.415191,104.274150,0.028940,1.347471,0.653218,...,866.166375,159.056561,2190.619005,641.435834,-2701.998961,-1496.659307,-3301.654393,-2910.654771,0.692269,-0.030766
4913,1610612757,21201230,48.520786,18.658315,36.381507,57.191997,126.172728,0.379570,1.502680,-0.035232,...,2074.187342,1151.247883,-710.791493,571.940697,523.350905,-610.349813,945.773586,-63.383898,1.631551,0.748113
4914,1610612744,21201230,9.678046,4.404986,21.860782,-19.101627,45.467029,0.552892,0.498168,0.918755,...,773.303151,-417.224507,990.195622,699.714084,150.916610,-1677.641396,-2417.270303,-3393.845387,-2.738317,-0.332978


## Loop through years and save data

In [ ]:
## merge rows

In [ ]:
import os
from glob import glob

In [ ]:
for i in glob('../*[0-9]'):
   %cd $i
   raw_scores = pd.read_csv("{}/raw_scores.txt".format(i))
   vegas = pd.read_csv("{}/vegas.txt".format(i))
   #  vegas_p = pd.read_csv("vegas_playoff.txt")

   # Preprocess DFs

   # Perturb each DF

   # Merge DFs for games

   # Save DFs
   

/root/nba-data/2018-19
['vegas_playoff.txt', 'raw_scores.txt', 'vegas.txt']
/root/nba-data/2012-13
['vegas_playoff.txt', 'raw_scores.txt', 'vegas.txt']
/root/nba-data/2013-14
['vegas_playoff.txt', 'raw_scores.txt', 'vegas.txt']
/root/nba-data/2014-15
['vegas_playoff.txt', 'raw_scores.txt', 'vegas.txt']
/root/nba-data/2017-18
['vegas_playoff.txt', 'raw_scores.txt', 'vegas.txt']
/root/nba-data/2016-17
['vegas_playoff.txt', 'raw_scores.txt', 'vegas.txt']
/root/nba-data/2015-16
['vegas_playoff.txt', 'raw_scores.txt', 'vegas.txt']
